In [ ]:
tree_path = '/content/drive/MyDrive/Interpretable Classifier Data/Decision Tree Model/tree.joblib'
model_path = '/content/drive/MyDrive/Interpretable Classifier Data/SemSeg Model/best_model.pt'
data_path = '/content/drive/MyDrive/Interpretable Classifier Data/Evaluation Data'
show_interpret = True

# Importing Libraries

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import os
import cv2
import numpy as np
from joblib import load
from model import ENet
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from utils import load_ckp, decode_segmap, counter, print_rule

# Loading Model

In [ ]:
model = ENet(num_classes=3).to('cuda')

In [ ]:
# define optimzer
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=2e-4)

# define checkpoint saved path
ckp_path = model_path

In [ ]:
model, optimizer, start_epoch, valid_loss_min = load_ckp(ckp_path, model, optimizer)

In [ ]:
clf = load(tree_path)

# Loading Data

In [ ]:
# Tranformation for the input images 
transform_img = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])

In [ ]:
class dataset(Dataset):
  """Custom Dataset class for ease of operation"""

  def __init__(self, images_X, images_Y, transform_img):
    self.data=images_X
    self.labels=images_Y
    self.transform=transform_img

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    x = self.data[index]
    y = self.labels[index]
    x = self.transform(x)
    return x, y

In [ ]:
image_list_X_test = []

In [ ]:
num_bicycles = len(os.listdir(data_path+'/bicycle/'))

for imagename in sorted(os.listdir(data_path+'/bicycle/')): 
  im=cv2.imread(data_path+'/bicycle/'+imagename)
  image_list_X_test.append(im)

In [ ]:
num_unicycles = len(os.listdir(data_path+'/unicycle/'))

for imagename in sorted(os.listdir(data_path+'/unicycle/')): 
  im=cv2.imread(data_path+'/unicycle/'+imagename)
  image_list_X_test.append(im)

In [ ]:
num_negative = len(os.listdir(data_path+'/negative/'))

for imagename in sorted(os.listdir(data_path+'/negative/')): 
  im=cv2.imread(data_path+'/negative/'+imagename)
  image_list_X_test.append(im)

In [ ]:
images_X_test = np.array(image_list_X_test)
y = [2]*num_bicycles + [1]*num_unicycles + [0]*num_negative

In [ ]:
test_data = dataset(images_X=images_X_test, images_Y=y, transform_img=transform_img)
test_loader = DataLoader(test_data, batch_size=1)

# Evaluation

In [ ]:
X_data = []
y_data = []

In [ ]:
for img, label in test_loader:

  model.eval()
  
  xb = img.to('cuda')

  yb = model(xb)

  pred = F.softmax(yb, dim=1)             
  idx = pred < 0.95
  pred[idx] = 0
  preds = torch.argmax(pred, dim=1).squeeze(1)

  preds = preds.cpu()

  x = decode_segmap(preds,nc=3)
  x = x.squeeze(0)

  temp = counter(x)

  if show_interpret:
    print_rule(np.array(temp).reshape(1,-1),clf)

  X_data.append(temp)
  y_data.append(int(label))

In [ ]:
X_data = np.array(X_data)

In [ ]:
y_pred = clf.predict(X_data)
accuracy_score(y_data, y_pred)*100

In [ ]:
print(confusion_matrix(y_data, clf.predict(X_data)))
print(classification_report(y_data, clf.predict(X_data)))